<a href="https://colab.research.google.com/github/rlsn/LungNoduleDetection/blob/main/nodule_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
# !pip install SimpleITK
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
# !rm -rf /content/LungNoduleDetection
!git clone https://github.com/rlsn/LungNoduleDetection.git
%cd /content/LungNoduleDetection
datadir="datasets/luna16"
!mkdir -p $datadir
! rm -rf $datadir
!ln -s /content/drive/MyDrive/datasets/luna16/ $datadir

Cloning into 'LungNoduleDetection'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 453 (delta 59), reused 64 (delta 25), pack-reused 342
Receiving objects: 100% (453/453), 673.30 MiB | 15.78 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/LungNoduleDetection


# View data

In [3]:
from dataset import LUNA16_Dataset, export_as_gif, collate_fn, compute_stats
import numpy as np
dataset = LUNA16_Dataset(data_dir=datadir,samples_per_img=4,return_bbox=True)
print(len(dataset))
re = dataset[887]
image = re["pixel_values"]
export_as_gif("ct.gif",image[0][0])
if len(re["bbox_imgs"])>0:
    export_as_gif("ct_marked.gif",re["bbox_imgs"][0])
re["pixel_values"].shape

888


torch.Size([4, 1, 40, 128, 128])

# Train

In [ ]:
from train import train
datadir="datasets/luna16"
train(datadir, log_dir="/content/drive/MyDrive/ColabLogs/luna-train", model_dir="/content/drive/MyDrive/ColabLogs/luna-train")

ViTConfig {
  "architectures": [
    "ViTDet3D"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "image_size": [
    40,
    128,
    128
  ],
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 6,
  "num_channels": 1,
  "num_hidden_layers": 4,
  "patch_size": [
    4,
    16,
    16
  ],
  "qkv_bias": true,
  "torch_dtype": "float32",
  "transformers_version": "4.38.2"
}

preparing datasets
preparing model
TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
data

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


commence training


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
